# Simulation V2
___


In [8]:
import numpy as np
import math
import random
import pygame
import time
import sys
import threading
from screeninfo import get_monitors

## Définition des variables globales :

In [9]:
m = get_monitors()[0]
screen_size = (m.width,m.height)
G = 16.67

## Définition des classes  :


In [10]:
class Planet:
    def __init__(self,mass,x,y,vx,vy,diametre,Fx = 0, Fy = 0,moove = True, color = (255,255,255)):
        self.mass = mass
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.diametre = mass//10 
        self.Fx = Fx
        self.Fy = Fy
        self.moove = moove
        self.color = color

class Button:
    def __init__(self,x,y,width,height,text,font_size = 30,text_color = (255,110,0),button_color = (200,200,200),view = True,status = True):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.rect = pygame.Rect(x,y,width,height)
        self.surface = pygame.Surface((width,height))
        self.text_color = text_color
        self.button_color = button_color
        self.font = pygame.font.Font(None, font_size)
        self.text = text
        self.status = status
    def show(self,screen):
        self.image_text = self.font.render(self.text,1,self.text_color)
        pygame.draw.rect(screen,self.button_color,self.rect)
        screen.blit(self.image_text, (self.x + (self.width - self.image_text.get_width())//2 , self.y + (self.height - self.image_text.get_height())//2 ))
        pygame.display.flip()
    def on_surface(self):
        x,y = pygame.mouse.get_pos()
        if (x >= self.x and x <= (self.x + self.width)) and (y >= self.y and y <= (self.y + self.height)):
            return True
        else:
            return False



## Calculs des mouvements :

In [11]:
def remoove(planet,p):
    tmp = []
    for pl in planet:
        if pl != p :
            tmp.append(pl)
    return tmp

In [12]:
def force(planets,cpt,planet_view):
    for p1 in planets:
        p1.Fx = 0
        p1.Fy = 0
        for i,p2 in enumerate(planets):
            if p1 != p2 :
                r = math.sqrt(pow((p1.x-p2.x),2) + pow((p1.y-p2.y),2))
                if (r**2*(p1.mass*p2.mass) <= 10000000000):                    
                    if r < (p1.diametre + p2.diametre)/1.5:
                        if p1.mass > p2.mass:
                            p1.Fx += p2.Fx
                            p1.Fy += p2.Fy
                            p1.mass += p2.mass*2
                            p1.diametre += math.sqrt(p2.diametre)
                            if i < cpt and planet_view : 
                                cpt -= 1 
                            planets = remoove(planets,p2).copy()
                            
                    p1.Fx += ((p2.x-p1.x)/r * G*(p1.mass*p2.mass)/r**2)
                    p1.Fy += ((p2.y-p1.y)/r * G*(p1.mass*p2.mass)/r**2)
                
    return planets,cpt



def coord(t,tor,screen_size,planets):
    for p in planets:
        if p.moove == True:
            if tor:
                p.x = (1/2*(p.Fx/(p.mass))*t**2+p.vx*t+p.x)%screen_size[0]
                p.y = (1/2*(p.Fy/(p.mass))*t**2+p.vy*t+p.y)%screen_size[1]
            else:
                p.x = (1/2*(p.Fx/(p.mass))*t**2+p.vx*t+p.x)
                p.y = (1/2*(p.Fy/(p.mass))*t**2+p.vy*t+p.y)
            p.vx = p.Fx/(p.mass)+p.vx
            p.vy = p.Fy/(p.mass)+p.vy   
            
            

## Définition des fonctions pour pygame:

In [13]:
def initialisation(nbr_planet,x1 = 0,x2 = 1920,y1 = 0,y2 = 1080):
    planets = []
    for i in range(nbr_planet):
        planets.append(Planet(random.randint(1,255), # Mass
                              random.randint(x1,x2), #Coord
                              random.randint(y1,y2),
                              random.randint(-50,50), #V0
                              random.randint(-50,50),
                              random.randint(5,10),)) # Diametre
    return planets

def pygame_init(screen_size):
    pygame.init()
    pygame.display.init()
    pygame.font.init()
    screen = pygame.display.set_mode(screen_size)
    pygame.display.set_caption("PLANET SIMULATOR")
    clock = pygame.time.Clock()
    return screen,clock

def menu(screen,clock,nbr_planet = 10,x1 = 0,x2 = m.width,y1 = 0,y2 = m.height):
    screen.fill((0,0,0))
    button1 = Button(200,200,250,20,str('Nombre de planètes : '+ str(nbr_planet)))
    button1_minus = Button(170,200,20,20,"-",font_size = 50,text_color = (255,255,255),button_color = (0,0,0))
    button1_plus = Button(475,197,20,20,"+",font_size = 50,text_color = (255,255,255),button_color = (0,0,0))
    Button.show(button1,screen)
    Button.show(button1_minus,screen)
    Button.show(button1_plus,screen)
    button_play = Button(200,100,100,20,"Play !")
    Button.show(button_play,screen)
    button_tor = Button(200,300,80,20,"tor")
    Button.show(button_tor,screen)
    button_size = Button(200,400,80,20,"normal")
    Button.show(button_size,screen)

    
    
    
    while(True):
        global planets
        clock.tick(100)          
        for event in pygame.event.get() :
            if event.type == pygame.QUIT :
                pygame.quit()
                return (False,0,0,0,0,0,0)
            if event.type == pygame.MOUSEBUTTONDOWN :
                if button1_plus.on_surface():
                    nbr_planet += 1
                    button1.text = 'Nombre de planètes : ' + str(nbr_planet)
                    Button.show(button1,screen)
                if button1_minus.on_surface():
                    nbr_planet -= 1
                    button1.text = 'Nombre de planètes : ' + str(nbr_planet)
                    Button.show(button1,screen)
                if button_play.on_surface():
                    planets = initialisation(x1,x2,y1,y2,nbr_planet)
                    if button_tor.status:
                        return (True, # Programm status
                                x1,x2,y1,x2,
                                True, #Tor status
                                nbr_planet
                                   )
                    else:
                        return (True, # Programm status
                                x1,x2,y1,x2,
                                False, # Tor status
                                nbr_planet
                               )
                    
                if button_tor.on_surface():
                    if button_tor.status:
                        button_tor.status = False
                        button_tor.text = "not tor"
                        Button.show(button_tor,screen)
                    else:
                        button_tor.status = True
                        button_tor.text = "tor"
                        Button.show(button_tor,screen)
                if button_size.on_surface():
                    if button_size.status:
                        button_size.status = False
                        button_size.text = "large"
                        Button.show(button_size,screen)
                        x1 = -4000
                        x2 = 4000
                        y1 = -2000 
                        y2 = 2000
                        
                    else:
                        button_size.status = True
                        button_size.text = "normal"
                        Button.show(button_size,screen)
                        x1 = 0
                        x2 = 1920
                        y1 = 0 
                        y2 = 1080

def simu(screen,clock,tor,planets): 
    zoom = 1
    ref = [0,0]
    cpt = 0
    skip = False
    planet_view = False
    while skip == False:
        if pygame.key.get_pressed()[pygame.K_SPACE]:
                ref = [m.width//2 - planets[cpt%len(planets)].x ,m.height//2 - planets[cpt%len(planets)].y]
                cpt += 1
                planet_view = True
                time.sleep(0.08)
                
        screen.fill((0,0,0)) 
        planets,cpt = force(planets,cpt,planet_view)
        coord(0.001,tor,screen_size,planets)
        for p in planets:
                x = p.x
                y = p.y
                if planet_view:
                         ref = [m.width//2 - planets[cpt%len(planets)].x/zoom ,m.height//2 - planets[cpt%len(planets)].y/zoom]
                pygame.draw.circle(screen, p.color,(x/zoom+ref[0],y/zoom+ref[1]),p.diametre/zoom + 1) 
                

                
     
        pygame.display.update()
        clock.tick(500)
        for event in pygame.event.get() :
            if event.type == pygame.QUIT :
                pygame.quit()
                return False
            if pygame.key.get_pressed()[pygame.K_ESCAPE]:
                return True

            if pygame.key.get_pressed()[pygame.K_KP_MINUS]:
                zoom += 1 
            if pygame.key.get_pressed()[pygame.K_KP_PLUS]:
                if zoom > 1 :
                    zoom -= 1




## Fonction principale :

In [14]:
def main():
    screen,clock = pygame_init(screen_size)
    status = True
    nbr_planet = 10
    while status:
        status,x1,x2,y1,y2,tor,nbr_planet = menu(screen,clock,nbr_planet)
        if status:
            planets = initialisation(nbr_planet,x1,x2,y1,y2)
            status = simu(screen,clock,tor,planets)
    return 0
main()

0